In [1]:
import sys  
sys.path.insert(0, '/Users/dami.osoba/work/bawk/models')

In [2]:
from  create_dataset import *
from train import trainIters
from predict import inference_from_file,evaluate
import torch
import librosa
import pickle
import os
from constants import max_duration


In [3]:
with open('../models/output/model_new/encoder_vars_new.pkl', 'rb') as convert_file:
    encoder = pickle.load(convert_file)

with open('../models/output/model_new/decoder_vars_new.pkl', 'rb') as convert_file:
    decoder = pickle.load(convert_file)

In [4]:
enc_path = '../models/output/model_new/enc_model_new'
encoder.load_state_dict(torch.load(enc_path))
encoder.eval()

dec_path = '../models/output/model_new/dec_model_new'
decoder.load_state_dict(torch.load(dec_path))
decoder.eval()

AttnDecoderRNN(
  (embedding): Embedding(29, 256)
  (attn): Linear(in_features=512, out_features=401, bias=True)
  (attn_combine): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (gru): GRU(256, 256)
  (out): Linear(in_features=256, out_features=29, bias=True)
)

In [6]:
import pandas
path ="/Users/dami.osoba/work/bawk/small_dataset/small/CV_unpacked/cv-corpus-6.1-2020-12-11/en/validated.tsv"
validated = pd.read_csv(path,sep='\t')

In [83]:
validated.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,0013037a1d45cc33460806cc3f8ecee9d536c45639ba4c...,common_voice_en_699711.mp3,She'll be all right.,2,1,NaN,NaN,NaN,en,NaN
1,0014c5a3e5715a54855257779b89c2bb498d470b225866...,common_voice_en_21953345.mp3,six,3,2,NaN,NaN,NaN,en,Benchmark
2,001509f4624a7dee75247f6a8b642c4a0d09f8be3eeea6...,common_voice_en_18132047.mp3,All's well that ends well.,2,0,NaN,NaN,NaN,en,NaN
3,003fb666a99eb3aa3ba05d9c8641c18e55cf7d34d1b981...,common_voice_en_17263741.mp3,Do you mean it?,2,0,NaN,NaN,NaN,en,NaN
4,004017ba82a23768d58dff3b91da8e8f951ea5fb6d3cd9...,common_voice_en_17893917.mp3,The new patch is less invasive than the old on...,2,1,NaN,NaN,NaN,en,NaN


In [7]:
fourth_pd = validated[validated['sentence'].astype('str')\
                      .apply(lambda x : any(a_string in x.split() for a_string in ['fourth','brain','hey']))]

In [8]:
fourth_pd.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment
65,04a9076ba20997a616bf98eb6d5f87ffaf6b4afd16d2f8...,common_voice_en_16327.mp3,He took the fourth part and handed it to the m...,4,0,NaN,NaN,NaN,en,NaN
2365,aaf5d369593020eb67196c5902f44a0db2c54d74bcd2a4...,common_voice_en_20529338.mp3,Both the third Viscount and his younger brothe...,2,1,NaN,NaN,NaN,en,NaN
5578,349a5bba6fb45860ea8530d5e19d5aa69f212beb5f2f84...,common_voice_en_530181.mp3,The human brain is still not fully understood.,2,0,NaN,NaN,NaN,en,NaN
6398,4b815c71216f4a1d37e4074c8e7d4effd7ec15376a0faf...,common_voice_en_21126145.mp3,The fourth side bears an inscription to the me...,2,0,NaN,NaN,NaN,en,NaN
8213,7aa3df30c263b19600f1d3bb33e96913f101833a63ab2f...,common_voice_en_19609644.mp3,"A fourth volume, ""The Blue Kremlin"", is projec...",2,0,twenties,female,us,en,NaN


In [9]:
fourth_list = fourth_pd['path'].tolist()

In [10]:
fourth_mp3 = [f"cv-corpus-6.1-2020-12-11/en/clips/{a}" for a in fourth_list[:20]]
fourth_wav = [f"/Users/dami.osoba/work/bawk/fourth_sample_wav/{a.replace('mp3','wav')}" for a in fourth_list]

In [11]:
for a in fourth_list:
    wav_file = a.replace('mp3','wav')
    os.system(
            f'ffmpeg -i /Users/dami.osoba/work/cv-corpus-6.1-2020-12-11/en/clips/{a} -ar 16000 /Users/dami.osoba/work/bawk/fourth_sample_wav/{wav_file} -y'
        )   

In [12]:
for wav in fourth_wav[:20]:
    trans = inference_from_file(wav, encoder, decoder,greedy=True)
    actuali =  fourth_pd.set_index('path').loc[wav.split('/')[-1].replace('wav','mp3')]['sentence']
    print(f"actual: {actuali}\n")

transcribe from file:  a didnt see any pointed by amelia
actual: He took the fourth part and handed it to the monk.

transcribe from file:  it is also widely used in the toptical media
actual: Both the third Viscount and his younger brother, the fourth Viscount, died childless.

transcribe from file:  but the contents failed to jell
actual: The human brain is still not fully understood.

transcribe from file:  the interface was indecipherable
actual: The fourth side bears an inscription to the memory of Clarkson.

transcribe from file:  he is nicknamed iron what theyre saidinaty been inderit
actual: A fourth volume, "The Blue Kremlin", is projected.

transcribe from file:  i am also will you herse following my destiny
actual: The fourth and fifth leading crops are sugar beets and hay.

transcribe from file:  now good weekend is edited by amelia lester
actual: It is the fourth oldest central bank in the world.

transcribe from file:  every daystoned we could be assainated
actual: Fenton

# pretrained model 

In [63]:
import nemo.collections.asr as nemo_asr
from nemo.collections.asr.metrics.wer import word_error_rate

[NeMo W 2021-09-09 18:10:29 optimizers:47] Apex was not found. Using the lamb optimizer will error out.
[NeMo W 2021-09-09 18:10:32 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [64]:
#better model
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")

[NeMo I 2021-09-09 18:10:50 cloud:56] Found existing object /Users/dami.osoba/.cache/torch/NeMo/NeMo_1.2.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
[NeMo I 2021-09-09 18:10:50 cloud:62] Re-using file from: /Users/dami.osoba/.cache/torch/NeMo/NeMo_1.2.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2021-09-09 18:10:50 common:676] Instantiating model from pre-trained checkpoint
[NeMo I 2021-09-09 18:10:51 features:252] PADDING: 16
[NeMo I 2021-09-09 18:10:51 features:269] STFT using torch
[NeMo I 2021-09-09 18:10:52 modelPT:438] Model EncDecCTCModel was successfully restored from /Users/dami.osoba/.cache/torch/NeMo/NeMo_1.2.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.


In [65]:
#worse model
asr_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_small_ls")

[NeMo I 2021-09-09 18:11:21 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_ctc_small_ls/versions/1.0.0/files/stt_en_conformer_ctc_small_ls.nemo to /Users/dami.osoba/.cache/torch/NeMo/NeMo_1.2.0/stt_en_conformer_ctc_small_ls/cf1b6bbcc08433257c12442c92b9996a/stt_en_conformer_ctc_small_ls.nemo
[NeMo I 2021-09-09 18:11:58 common:676] Instantiating model from pre-trained checkpoint
[NeMo I 2021-09-09 18:11:59 mixins:149] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2021-09-09 18:11:59 modelPT:139] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/Librispeech_SP_Tarred/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/Librispeech_SP_Tarred/audio__OP_0..511_CL_.tar
    
[NeMo W 2021-09-09 18:11:59 modelPT:146] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/librivox-dev-other.json
    - /manife

[NeMo I 2021-09-09 18:11:59 features:252] PADDING: 0
[NeMo I 2021-09-09 18:11:59 features:269] STFT using torch
[NeMo I 2021-09-09 18:12:00 modelPT:438] Model EncDecCTCModelBPE was successfully restored from /Users/dami.osoba/.cache/torch/NeMo/NeMo_1.2.0/stt_en_conformer_ctc_small_ls/cf1b6bbcc08433257c12442c92b9996a/stt_en_conformer_ctc_small_ls.nemo.


In [80]:
for wav in fourth_wav[:10]:
    trans = asr_model.transcribe(paths2audio_files=[wav])
    actuali =  fourth_pd.set_index('path').loc[wav.split('/')[-1].replace('wav','mp3')]['sentence']
    print("predicted:",trans[0],"\n",f"actual: {actuali}\n")

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

predicted: he took the boot t and and it thlew the muck 
 actual: He took the fourth part and handed it to the monk.



Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

predicted: both the third viscount and his younger brother the fourth viscount died childless 
 actual: Both the third Viscount and his younger brother, the fourth Viscount, died childless.



Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

predicted: freandon was the fourth of seven gogen by his father's first marriage 
 actual: Fenton was the fourth of seven children by his father's first marriage.



Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

predicted: manimore fled the country leaving toi a brain drain 
 actual: Many more fled the country, leading to a brain drain.



Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

predicted: peter was born in bright slower the fth son of stance slw and franticcar 
 actual: Peter was born in Bratislava, the fourth son of Stanislav and Frantiska.



In [79]:
# hypothesis = asr_model.transcribe(paths2audio_files=files[:100])
# better model 
for wav in fourth_wav[:10]:
    trans = quartznet.transcribe(paths2audio_files=[wav])
    actuali =  fourth_pd.set_index('path').loc[wav.split('/')[-1].replace('wav','mp3')]['sentence']
    print("predicted:",trans[0],"\n",f"actual: {actuali}\n")

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2021-09-09 18:16:40 nemo_logging:349] /Users/dami.osoba/opt/anaconda3/envs/tf/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
    To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459064158/work/aten/src/ATen/native/BinaryOps.cpp:467.)
      return torch.floor_divide(self, other)
    


predicted: he took the forte up and handed to the muck 
 actual: He took the fourth part and handed it to the monk.



Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

predicted: both the third viscount and his younger brother e forth discount died childless 
 actual: Both the third Viscount and his younger brother, the fourth Viscount, died childless.



...

predicted: fanton was the fourth of seven children bis fathers first marriage 
 actual: Fenton was the fourth of seven children by his father's first marriage.



Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

predicted: manimore fled the country leading to a brain drain 
 actual: Many more fled the country, leading to a brain drain.



Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

predicted: peter was born in bradslawar the fourth son of stanilav and frandisca 
 actual: Peter was born in Bratislava, the fourth son of Stanislav and Frantiska.



In [78]:
#get word error rate
wer = word_error_rate(hypotheses=["manimore fled the country leaving toi a brain drain"], 
                      references=["Many more fled the country, leading to a brain drain"])
print(wer)

0.5
